In [0]:
from keras.layers import Dense, Embedding, Conv1D
from keras.layers import Flatten, Dropout
from keras.optimizers import Adam
from keras.models import Model, Sequential
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from datetime import timedelta

import time
import pandas as pd

In [0]:
start_time = time.time()
print(start_time)
print('Training Word2Vec Model...')

data = pd.read_csv('Hasil_Sentimen.csv',index_col=None)


vectorizer = CountVectorizer()

x = vectorizer.fit_transform(data['tweets'].values)
y = data['label'].values

trainX, testX, trainy, testy = train_test_split(x, y, test_size=0.3)

# this returns a tensor
print("Creating Model...")
verbose, epochs, batch_size = 1, 100, 250

n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], trainy.shape[0]
model = Sequential()
model.add(Embedding(n_timesteps, batch_size, input_length=n_features))
model.add(Conv1D(10,(5), activation='relu', padding='same'))
model.add(Conv1D(10,(5), activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(60, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Traning Model...")
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data=(testX,testy))
	
finish_time = time.time()

model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

print('Finished. Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))